In [0]:
import pandas as pd

df = pd.read_csv(
    "/Volumes/workspace/default/files/ClassicHit.csv"
)
#display(df)

# Pegar esse arquivo e levar para a camada Bronze.
spark.sql(
    "TRUNCATE TABLE workspace.bronze.classic_hit"
)

spark_df = spark.createDataFrame(df)

spark_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.bronze.classic_hit")



In [0]:
import requests
import base64
import pandas as pd

# Funcao para conseguir o token para utilizar as requests da API
def get_spotify_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(
            f"{client_id}:{client_secret}".encode()
        ).decode()
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get token: {response.text}")

# Função para buscar informações de uma música no Spotify
def get_spotify_track_info(track_name, artist_name, token):
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": f"track:{track_name} artist:{artist_name}",
        "type": "track",
        "limit": 1
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
    
        items = response.json()["tracks"]["items"]

        if items:
            return {
                "return": items
            }
            
        return None

# Usa sua client_id e client_secret aqui, para autenticacao da API
client_id = "50838a69a5fc445b86d340b4b97ef5b4"
client_secret = "21932daca2264477aa9bda20c037c6e2"
token = get_spotify_access_token(client_id, client_secret)

track = get_spotify_track_info("A Beautiful Morning", "The Rascals", token)

print(track)

# Interindo a informacao da API do Spotify no Pandas DataFrame
df = pd.DataFrame([track])
display(df)
